<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon-2022/blob/main/Notebooks2022/build_a_thon_graph_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created: 14 Apr 2022.   
Contact: Vishnu Ram OV.   
vishnu.n@ieee.org.   
Licence: available for all purposes under the sun


Refer: https://neo4j.com/docs/aura/auradb/getting-started/create-database/

step-1: install neo4j

In [1]:
pip install neo4j


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.8 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.3-py3-none-any.whl size=116069 sha256=8f86de972ec6a9d0c6ad174f3f5a9b557eb4b0e8a4e62594b51c6a6448df50df
  Stored in directory: /root/.cache/pip/wheels/db/dd/76/acacd519878f133f2f869aec70db548d89e04013209c3c62bc
Successfully built neo4j


step-2: imports

In [2]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

step-3: define helper class

In [20]:


class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    #Vishnu: 1 Jun 2022: added relationships names
    #Vishnu: CAUTION: side-effect: this f() creates the nodes
    def create_nodes_with_friendship(self, actor1_name, actor2_name, rel_name):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_friendship, actor1_name, actor2_name, rel_name)
            for row in result:
                print("Created friendship between: {n1}, {n2}".format(n1=row['n1'], n2=row['n2']))
    
    #Vishnu: 1 Jun 2022: added relationships names
    #Vishnu: CAUTION: side-effect: this f() creates the nodes
    #NOTE: we use labels and properties for nodes
    #NOTE: we use relationtypes only 
    #TBD - to search for existing nodes and reuse them.
    #TBD - add properties to relationships

    @staticmethod
    def _create_and_return_friendship(tx, actor1_name, actor2_name, rel_name):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "CREATE (n1:Actor { name: $actor1_name }) "
            "CREATE (n2:Actor { name: $actor2_name }) "
            "CREATE (n1)-[:" + rel_name + "]->(n2) "
            "RETURN n1, n2"
        )
        result = tx.run(query, actor1_name=actor1_name, actor2_name=actor2_name)
        try:
            return [{"n1": row["n1"]["name"], "n2": row["n2"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_all_actors(self):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_actors)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    @staticmethod
    def _find_and_return_all_actors(tx):
        query = (
            "MATCH (n:Actor) "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

step-4: connect to DB

In [21]:
#Vishnu: 1 Jun 2022: use the details from https://console.neo4j.io
#especially use your own DB instance and corresponding passwd.

uri = "neo4j+s://a4270446.databases.neo4j.io"
user = "neo4j"
password = "secret"

app = App(uri, user, password)

Step-5: Create nodes and relations

In [25]:
app.create_nodes_with_friendship("Experimentation Controller", "Sandbox", "my_best_buddy")


Created friendship between: Experimentation Controller, Sandbox


Step-6: Find all actors

In [26]:
app.find_all_actors()

Found actor-1: Experimentation Controller
Found actor-2: Sandbox


Step-6: Close the database connection

In [7]:
app.close()